In [1]:
import optuna
from optuna.trial import TrialState
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from dataloader import load_data, preprocess_graph
from models.SGCRes import SGCRes
from models.SSGC import SSGC
from models.DGC import DGC
from models.SGC import SGC
from stacked_models import DeepLinear, LinearMLP
from models.GCN import GCN
from dgl.nn.pytorch.conv import SGConv
from time import perf_counter

d:\Files\Coding\ml\EfficientGNN\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def objective(trial):
    
    graph, labels, split_idx = load_data('cora')
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    g = preprocess_graph(graph).to(device)
    labels=labels.to(device)
    # print(graph.ndata['feat'])
    features = graph.ndata['feat'].to(device)

    in_feats = features.shape[-1]
    n_classes = labels.max().item()+1
    #K=2
    K=trial.suggest_int('K',2,20,log=True)
    is_linear=False
    loss_fn=nn.CrossEntropyLoss()
    model=SGC(in_feats,n_classes,K,device,is_linear=True)
    epochs=100
    epochs=trial.suggest_int('epochs',50,100)
    train_mask,val_mask=split_idx["train_mask"], split_idx["val_mask"]
    #lr=0.2
    lr=trial.suggest_float("lr",1e-5,1e-3)
    #weight_decay=1.2e-05
    weight_decay=trial.suggest_float('weight_decay',1e-6,1e-3,log=True)
    
    t = perf_counter()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    train_mask=train_mask.to(device)
    val_mask=val_mask.to(device)
    labels=labels.to(device)
    g=g.to(device)
    features=features.to(device)
    for epoch in range(epochs):
        model.train()
        logits = None
        if is_linear:
            logits = model(features)
        else:
            logits = model(g, features)
        loss = loss_fn(logits[train_mask], labels[train_mask])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        model.eval()
        if is_linear:
            logits = model(features)
        else:
            logits = model(g, features)
        train_acc = torch.sum(logits[train_mask].argmax(1) == labels[train_mask]).item() / train_mask.sum().item()
        val_acc = torch.sum(logits[val_mask].argmax(1) == labels[val_mask]).item() / val_mask.sum().item()
        # test_acc = torch.sum(logits[test_mask].argmax(1) == labels[test_mask]).item() / test_mask.sum().item()
        # print(f'Epoch {epoch + 1:02d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, Val: {val_acc:.4f}')
        trial.report(val_acc,epoch)
    training_time = perf_counter()-t
    print(f'Training time: {training_time:.4f}s')
    return val_acc

In [3]:
study = optuna.create_study(direction="maximize")

study.optimize(objective, n_trials=100, timeout=600)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])


[I 2022-12-20 08:37:00,036] A new study created in memory with name: no-name-c048d636-1fa7-473f-bcde-905887e8dedb


  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


[I 2022-12-20 08:37:05,380] Trial 0 finished with value: 0.766 and parameters: {'K': 4, 'epochs': 91, 'lr': 0.0007884632204204343, 'weight_decay': 0.00011043671488265508}. Best is trial 0 with value: 0.766.


Training time: 5.2237s
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


[I 2022-12-20 08:37:16,991] Trial 1 finished with value: 0.772 and parameters: {'K': 7, 'epochs': 73, 'lr': 0.0006901061237451926, 'weight_decay': 0.0004881072005732666}. Best is trial 1 with value: 0.772.


...

Training time: 2.5083s
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


[I 2022-12-20 08:43:25,466] Trial 98 finished with value: 0.772 and parameters: {'K': 7, 'epochs': 96, 'lr': 0.0008226858780151861, 'weight_decay': 2.785758155438177e-05}. Best is trial 20 with value: 0.79.


Training time: 2.5438s
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


[I 2022-12-20 08:43:27,974] Trial 99 finished with value: 0.774 and parameters: {'K': 11, 'epochs': 86, 'lr': 0.0008999118749700648, 'weight_decay': 1.1981165072884218e-06}. Best is trial 20 with value: 0.79.


Training time: 2.3140s
Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  0
  Number of complete trials:  100
Best trial:
  Value:  0.79
  Params: 
    K: 8
    epochs: 61
    lr: 0.0008751456703795927
    weight_decay: 1.0080744555697913e-06


In [ ]:
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [4]:
optuna.visualization.plot_param_importances(study)
optuna.visualization.plot_contour(study)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'type': 'scatter', 'xaxis': 'x', 'yaxis': 'y'},
             {'colorbar': {'title': {'text': 'Objective Value'}},
              'colorscale': [[0.0, 'rgb(247,251,255)'], [0.125,
                             'rgb(222,235,247)'], [0.25, 'rgb(198,219,239)'],
                             [0.375, 'rgb(158,202,225)'], [0.5,
                             'rgb(107,174,214)'], [0.625, 'rgb(66,146,198)'],
                             [0.75, 'rgb(33,113,181)'], [0.875, 'rgb(8,81,156)'],
                             [1.0, 'rgb(8,48,107)']],
              'connectgaps': True,
              'contours': {'coloring': 'heatmap'},
              'hoverinfo': 'none',
              'line': {'smoothing': 1.3},
              'reversescale': False,
              'showscale': True,
              'type': 'contour',
              'x': [1.787079263164137, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,
                    15, 17, 19, 21.26374626087854],
              'xaxis': 'x5',
              'y': [47.5, 50, 53, 55, 56, 58, 59, 60, 61, 62, 63, 65, 66, 67, 68,
                    69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 80, 81, 83, 84, 85, 86,
                    87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 102.5],
              'yaxis': 'y5',
              'z': [[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
                    nan, nan, nan, nan, nan, nan], [nan, nan, nan, nan, 0.744, nan,
                    0.752, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
                    [nan, nan, nan, 0.738, nan, nan, nan, nan, nan, nan, nan, nan,
                    nan, nan, nan, nan, 0.684, nan], [nan, nan, nan, nan, nan, nan,
                    nan, nan, nan, nan, nan, nan, 0.734, 0.764, nan, nan, nan,
                    nan], [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
                    nan, nan, 0.768, nan, nan, nan, nan], [nan, nan, nan, nan, nan,
                    nan, nan, nan, nan, nan, 0.74, nan, nan, nan, nan, nan, nan,
                    nan], [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
                    nan, nan, nan, nan, nan, 0.718, nan], [nan, nan, nan, nan, nan,
                    nan, 0.38, nan, nan, 0.768, 0.77, nan, nan, nan, nan, nan, nan,
                    nan], [nan, nan, nan, nan, nan, nan, nan, 0.79, nan, nan, nan,
                    nan, nan, nan, nan, nan, nan, nan], [nan, nan, nan, nan, nan,
                    nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.726, nan,
                    nan], [nan, nan, 0.722, nan, nan, nan, nan, nan, nan, nan, nan,
                    nan, nan, nan, nan, nan, nan, nan], [nan, nan, 0.768, nan,
                    0.762, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
                    nan, nan], [nan, nan, nan, nan, nan, nan, nan, nan, 0.752, nan,
                    nan, nan, nan, nan, nan, nan, nan, nan], [nan, nan, 0.732, nan,
                    nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
                    nan, nan], [nan, nan, nan, nan, nan, nan, nan, nan, 0.74, nan,
                    nan, nan, nan, nan, nan, nan, nan, nan], [nan, nan, nan, nan,
                    0.724, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
                    nan, nan], [nan, nan, nan, nan, nan, nan, 0.774, nan, nan, nan,
                    nan, nan, nan, nan, nan, nan, nan, nan], [nan, nan, nan, nan,
                    nan, nan, nan, nan, 0.766, nan, nan, nan, nan, nan, nan, nan,
                    nan, nan], [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
                    0.758, nan, nan, nan, nan, nan, nan, nan], [nan, nan, nan, nan,
                    nan, nan, 0.772, nan, nan, nan, nan, nan, nan, nan, nan, nan,
                    nan, nan], [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
                    0.762, 0.768, nan, nan, nan, nan, nan, nan], [nan, nan, 0.736,
                    nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.756, nan, nan,
            